In [1]:
from z3 import *

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from IPython.display import Image

from decisiontree.dt_goal_recogniser import HandcraftedGoalTrees, TrainedDecisionTrees
from evaluation.verification import add_goal_tree_model, extract_counter_example
from core.data_processing import get_dataset
from core.feature_extraction import FeatureExtractor
from core.scenario import ScenarioConfig, Scenario
from core.base import get_data_dir, get_scenario_config_dir, get_img_dir
from core.lanelet_helpers import LaneletHelpers

## Handcrafted goal tree

In [2]:
scenario_name = 'heckstrasse'
model = HandcraftedGoalTrees.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'turn-left')]

s = Solver()

features, probs = add_goal_tree_model(reachable_goals, s, model)

# unsatisfiable if G2 always has highest prob
verify_expr = Implies(And(features[1]['in_correct_lane'], Not(features[2]['in_correct_lane'])), probs[2] < probs[1])
s.add(Not(verify_expr))

print(s.check())


unsat


verified true:
If car is in correct lane for G1, then G1 is predicted

In [3]:
scenario_name = 'heckstrasse'
model = HandcraftedGoalTrees.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'turn-left')]

s = Solver()

features, probs = add_goal_tree_model(reachable_goals, s, model)

verify_expr = Implies(And(Not(features[1]['in_correct_lane']),
                          features[2]['in_correct_lane']), probs[2] > probs[1])
s.add(Not(verify_expr))

print(s.check())

unsat


Verified true:
If car is in correct lane for G2, G2 is predicted


# Trained goal tree

In [4]:
scenario_name = 'heckstrasse'
model = TrainedDecisionTrees.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'turn-left')]

s = Solver()

features, probs = add_goal_tree_model(reachable_goals, s, model)

verify_expr = Implies(And(features[1]['in_correct_lane'],
                          Not(features[2]['in_correct_lane'])),
                      probs[2] < probs[1])
s.add(Not(verify_expr))

print(s.check())

sat


In [5]:
s.model()

[likelihood_1_straight-on = 3328868578632209/62500000000000000,
 angle_in_lane_1 = 1/32,
 likelihood_2_turn-left = 4733690513709423/5000000000000000,
 path_to_goal_length_2 = 611815071105957/10000000000000,
 acceleration_1 = 183992999792099/200000000000000,
 path_to_goal_length_1 = 783497200012207/10000000000000,
 in_correct_lane_2 = False,
 in_correct_lane_1 = True,
 prob_2_turn-left = 69741296702901268516935306473415/92669415458121270862886033690107,
 prob_1_straight-on = 22928118755220002345950727216692/92669415458121270862886033690107,
 vehicle_in_front_speed_1 = 0,
 vehicle_in_front_speed_2 = 0,
 vehicle_in_front_speed = 0,
 vehicle_in_front_dist_1 = 0,
 vehicle_in_front_dist_2 = 0,
 vehicle_in_front_dist = 0,
 angle_in_lane_2 = 1/32,
 angle_in_lane = 1/32,
 acceleration_2 = 183992999792099/200000000000000,
 acceleration = 183992999792099/200000000000000,
 speed_1 = 0,
 speed_2 = 0,
 speed = 0,
 /0 = [(41844778021740761110161183884049/500000000000000000000000000000000,
        278008246374363812588658101070321/2500000000000000000000000000000000) ->
       69741296702901268516935306473415/92669415458121270862886033690107,
       (17196089066415001759463045412519/625000000000000000000000000000000,
        278008246374363812588658101070321/2500000000000000000000000000000000) ->
       22928118755220002345950727216692/92669415458121270862886033690107,
       else -> 0]]

In [6]:
extract_counter_example(s, features)

TypeError: float() argument must be a string or a number, not 'NoneType'

In [ ]:
# G1
22928118755220002345950727216692/76318343603113298254273810502177

In [ ]:
# G2
53390224847893295908323083285485/76318343603113298254273810502177

Verification failed: If car is in correct lane for G1, then G1 is predicted.
Angle in lane > 0.08

## Investigate examples of this

In [ ]:
Image("../images/trained_tree_heckstrasse_G1_straight-on.png")

In [ ]:
Image("../images/trained_tree_heckstrasse_G2_turn-left.png")

In [ ]:
dataset = get_dataset(scenario_name, 'train')

In [ ]:
scenario = Scenario.load(get_scenario_config_dir() + '{}.json'.format(scenario_name))

In [ ]:
feature_extractor = FeatureExtractor(scenario.lanelet_map)

In [ ]:
episode = scenario.load_episode(0)

In [ ]:
state = episode.frames[8292].agents[114]

In [ ]:
goal_routes = feature_extractor.get_goal_routes(state, scenario.config.goals)

In [ ]:
plt.figure(figsize=(12,8))
scenario.plot()
LaneletHelpers.plot(goal_routes[1].shortestPath()[0])

In [ ]:
plt.figure(figsize=(12,8))
scenario.plot()
LaneletHelpers.plot(goal_routes[2].shortestPath()[0])

In [ ]:
dataset.loc[(dataset.in_correct_lane)  
            & (dataset.angle_in_lane > 0.05)
           & (dataset.possible_goal==1) & (dataset.goal_type=='straight-on')]

Reasons for wrong lane:
1. Start of trajectory, has not yet changed lane

heckstrasse agent 114 frame 8292 get current lanelet?


In [ ]:
scenario_name = 'heckstrasse'
model = TrainedDecisionTrees.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'turn-left')]

s = Solver()

features, probs = add_goal_tree_model(reachable_goals, s, model)


verify_expr = Implies(And(Not(features[1]['in_correct_lane']), features[2]['in_correct_lane']), probs[2] > probs[1])
s.add(Not(verify_expr))

print(s.check())

Verified true: If car is in correct lane for G2, G2 is predicted

# Verify bound on probability if in the correct lane
If we are in the correct lane for a goal, the the probabilty of that goal should be > 0.2
Heckstrasse - coming from west

In [ ]:
scenario_name = 'heckstrasse'
model = TrainedDecisionTrees.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'turn-left')]

s = Solver()

features, probs = add_goal_tree_model(reachable_goals, s, model)


verify_expr = Implies(And(features[1]['in_correct_lane'],
                          Not(features[2]['in_correct_lane'])), probs[1] >= 0.2)
s.add(Not(verify_expr))

print(s.check())

In [ ]:
scenario_name = 'heckstrasse'
model = TrainedDecisionTrees.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'turn-left')]

s = Solver()

features, probs = add_goal_tree_model(reachable_goals, s, model)


verify_expr = Implies(And(Not(features[1]['in_correct_lane']),
                          features[2]['in_correct_lane']), probs[2] >= 0.2)
s.add(Not(verify_expr))

print(s.check())

For both goals, this is verified to be true

# Verify prob based on angle in lane, approaching T Junction

In [ ]:
scenario_name = 'heckstrasse'
model = TrainedDecisionTrees.load(scenario_name)
reachable_goals = [(0, 'turn-right'), (1, 'turn-left')]

s = Solver()

features, probs = add_goal_tree_model(reachable_goals, s, model)

s.add(features[0]['angle_in_lane'] >= np.pi / 8)
s.add(features[1]['angle_in_lane'] >= np.pi / 8)

verify_expr =  probs[1] >= 0.1
s.add(Not(verify_expr))

print(s.check())

In [ ]:
2346984033116499/62500000000000000

In [ ]:
52939762064801009122130908858125/55512532424702459568794311420649

In [ ]:
2572770359901450446663402562524/55512532424702459568794311420649

In [ ]:
Image("../images/trained_tree_heckstrasse_G0_turn-right.png")

In [ ]:
2346984033116499/62500000000000000

In [ ]:
Image("../images/trained_tree_heckstrasse_G1_turn-left.png")

In [ ]:
6231454005934719/10000000000000000

In [ ]:
model.goal_priors

# Verify that decreasing path to goal length leads to decreasing entropy
Can easily be done when there are two goals
Start with heckstrasse approaching from west

In [ ]:
scenario_name = 'heckstrasse'
model = TrainedDecisionTrees.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'turn-left')]

s = Solver()

# 2 has shorter path to goal length than 1, but all other features are equal
features1, probs1 = add_goal_tree_model(reachable_goals, s, model, suffix='_1')
features2, probs2 = add_goal_tree_model(reachable_goals, s, model, suffix='_2')

for goal_idx, goal_type in reachable_goals:
    s.add(features1[goal_idx]['path_to_goal_length'] \
          > features2[goal_idx]['path_to_goal_length'])
    
    for feature_name in features1[goal_idx]:
        if feature_name != 'path_to_goal_length':
            s.add(features1[goal_idx][feature_name] == features2[goal_idx][feature_name])


verify_expr = And(Implies(probs1[1] < probs1[2], probs2[2] >= probs1[2]), 
                  Implies(probs1[1] > probs1[2], probs2[1] >= probs1[1]))

s.add(Not(verify_expr))


print(s.check())

In [ ]:
s.model()

verification failed
Counterexample:
path_to_goal_length_1_1 = -1/4
path_to_goal_length_1_2 = -2

This features caused the difference:
path_to_goal_length_2_1 = 62
path_to_goal_length_2_2 = 59

likelihood_2_turn-left_1 = 0.032
likelihood_2_turn-left_2 = 0.002

likelihood_1_straight-on_1 = 0.0003
likelihood_1_straight-on_2 = 0.0003

in_correct_lane is false for both goals - should not be possible

In [ ]:
prob_2_turn_left_2 = 12985641854098344188989774327125/24583179829227809648801218976353
prob_1_straight_on_2 = 11597537975129465459811444649228/24583179829227809648801218976353
prob_2_turn_left_1 = 46699757191188387508825432121897/49599141684970753873778293284204
prob_1_straight_on_1 = 2899384493782366364952861162307/49599141684970753873778293284204



In [ ]:
print(prob_1_straight_on_1)
print(prob_1_straight_on_2)
print(prob_2_turn_left_1)
print(prob_2_turn_left_2)

In [ ]:
1683813668581631/5000000000000000000

## Try verification again, restricting feature values based on domain knowledge

Eventually domain model / feature extraction could be represented in logic

In [ ]:
scenario_name = 'heckstrasse'
model = TrainedDecisionTrees.load(scenario_name)
reachable_goals = [(1, 'straight-on'), (2, 'turn-left')]

s = Solver()

# 2 has shorter path to goal length than 1, but all other features are equal
features1, probs1 = add_goal_tree_model(reachable_goals, s, model, suffix='_1')
features2, probs2 = add_goal_tree_model(reachable_goals, s, model, suffix='_2')

for goal_idx, goal_type in reachable_goals:
    s.add(features1[goal_idx]['path_to_goal_length'] \
          > features2[goal_idx]['path_to_goal_length'])
    
    for feature_name in features1[goal_idx]:
        if feature_name != 'path_to_goal_length':
            s.add(features1[goal_idx][feature_name] == features2[goal_idx][feature_name])

s.add(features1[1]['in_correct_lane'] != features1[2]['in_correct_lane'])

            
verify_expr = And(Implies(probs1[1] < probs1[2], probs2[2] == probs1[2]), 
                  Implies(probs1[1] > probs1[2], probs2[1] == probs1[1]))

s.add(Not(verify_expr))


print(s.check())

In [ ]:
s.model()

In [ ]:
likelihood_2_turn_left_2 = 35256017633877/16000000000000000
likelihood_1_straight_on_2 = 28828658074298713/100000000000000000
likelihood_2_turn_left_1 = 15848730096759557/500000000000000000
likelihood_1_straight_on_1 = 28828658074298713/100000000000000000

In [ ]:

print(likelihood_1_straight_on_1)
print(likelihood_1_straight_on_2)
print(likelihood_2_turn_left_1)
print(likelihood_2_turn_left_2)


Not verified - different child node reached  depending path to goal length
When further from goal, acceleration is informative - high acceleration means lower G2 likelihood

Looks like overfitting - see tree below

In [ ]:
Image('../images/trained_tree_heckstrasse_G2_turn-left.png')